# Trabalhando com Permissoes por meio de Role, Grant e Revoke

### Conectando na base de dados e verificando roles existentes

In [1]:
import psycopg2
import pandas as pd
conexao = psycopg2.connect(host="localhost",
                                database="empresa", 
                                user="postgres", 
                                password="123456",
                                port=5432)
res = pd.read_sql_query("""SELECT rolname FROM pg_roles""",conexao)
conexao.close()
res

,rolname
0,pg_monitor
1,pg_read_all_settings
2,pg_read_all_stats
3,pg_stat_scan_tables
4,pg_signal_backend
5,postgres


## Criando função de gerenciamento de conexao e execucao de códigos com postgres

#### O objetivo da função abaixo inclui executar todo processo de controle de erros de código SQL e gerenciamento de conexão, de modo que possamos manter o foco nas novas instruções que serão apresentadas. Observe que somente o Status da execução é retornado, possibilitando avaliar se a instrução foi executada e ter uma ideia do resultado.

In [2]:
def execute_code_postgres(codigo_sql):
    import psycopg2
    import pandas as pd
    conexao = None
    try:
        """ Connect to the PostgreSQL database server """
        conexao = psycopg2.connect(host="localhost",
                                database="empresa", 
                                user="postgres", 
                                password ="123456",   
                                port=5432)
        cursor_elephant = conexao.cursor()
        cursor_elephant.execute(codigo_sql)
        # close the communication with the PostgreSQL
        print(cursor_elephant.statusmessage)
        cursor_elephant.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        if conexao is not None:
            conexao.rollback()
        print(""" ==================================""")
        print(""">>>>>>>   Processing ERROR   <<<<<<""")
        print(""" ==================================""")
    else:
        if conexao is not None:
            conexao.commit()
        print(""">>>>>>>    Processing performed correctly     <<<<<<""")
        
    finally:
        if conexao is not None:
            conexao.close()
            print('Database connection closed.')
            

#### Ao executar a função "execute_code_postgres" o somente será necessário enviar a instrução a ser executada. Observe que além de informar se tudo ocorreu corretamente ou com algum erro, o conjunto de instruções cursor_elephant.statusmessage possibilita verificar qual foi a instrução executada.  

In [3]:
execute_code_postgres("""select * from empregado""")

SELECT 8
>>>>>>>    Processing performed correctly     <<<<<<
Database connection closed.


# Criando Novo usuário no banco de dados.

### CREATE ROLE role_name;
Possibilita criar um grupo ou usuário de acesso ao sistema.

In [4]:
execute_code_postgres(""" CREATE ROLE aluno01
""")

CREATE ROLE
>>>>>>>    Processing performed correctly     <<<<<<
Database connection closed.


## Criando função de gerenciamento de conexao e execucao de códigos com usuário criado

In [5]:
def execute_code_aluno01(codigo_sql):
    import psycopg2
    import pandas as pd
    conexao = None
    try:
        """ Connect to the PostgreSQL database server """
        conexao = psycopg2.connect(host="localhost",
                                database="empresa", 
                                user="aluno01", 
                                password ="101010",   
                                port=5432)
        cursor_elephant = conexao.cursor()
        cursor_elephant.execute(codigo_sql)
        # close the communication with the PostgreSQL
        print(cursor_elephant.statusmessage)
        cursor_elephant.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        if conexao is not None:
            conexao.rollback()
        print(""" ==================================""")
        print(""">>>>>>>   Processing ERROR   <<<<<<""")
        print(""" ==================================""")
    else:
        if conexao is not None:
            conexao.commit()
        print(""">>>>>>>    Processing performed correctly     <<<<<<""")
        
    finally:
        if conexao is not None:
            conexao.close()
            print('Database connection closed.')
            

## Tentativa de login com usuário aluno01 para acesso de informações do sistema

In [6]:
execute_code_aluno01("""select * FROM pg_database""")

FATAL:  password authentication failed for user "aluno01"
FATAL:  password authentication failed for user "aluno01"

>>>>>>>   Processing ERROR   <<<<<<


> Observe que a role criada não possui permissão de login para autenticar no sistema. Precisamos então atribuir tais permissões.

# Alterando uma role 

ALTER ROLE name [ [ WITH ] option [ ... ] ]

Algumas opções que podem ser utilizadas:

      SUPERUSER | NOSUPERUSER
    | CREATEDB | NOCREATEDB
    | CREATEROLE | NOCREATEROLE
    | CREATEUSER | NOCREATEUSER
    | LOGIN | NOLOGIN
    | CONNECTION LIMIT connlimit
    | PASSWORD 'password'
    | VALID UNTIL 'timestamp'

## Alterando role e adicionando permissões de login

In [7]:
execute_code_postgres(""" alter role aluno01 with login password '101010'
""")

ALTER ROLE
>>>>>>>    Processing performed correctly     <<<<<<
Database connection closed.


> Neste caso atribuimos ao usuario criado a permissão de login no sistema

## Tentativa de login com usuário aluno01 para acesso de informações do sistema SGBD

### Verificando bancos de dados existentes no sistema

In [8]:
execute_code_aluno01("""select * FROM pg_database""")

SELECT 30
>>>>>>>    Processing performed correctly     <<<<<<
Database connection closed.


In [9]:
execute_code_aluno01("""select * FROM pg_tables where schemaname='public'""")

SELECT 27
>>>>>>>    Processing performed correctly     <<<<<<
Database connection closed.


## Tentativa de acess a uma tabela específica de um determinado banco de dados do sistema

In [10]:
execute_code_aluno01("""select * from empregado""")

permission denied for relation empregado

>>>>>>>   Processing ERROR   <<<<<<
Database connection closed.


> isto aconteceu, pois apesar de atribuirmos um login ao aluno01 não definimos permissões de acesso para ele para a tabela desejada

# Utilizando Grant e Revoke para atribuir permissões

Depois de atribuir permissões de login a um novo usuário é possível "logar" no banco de dados

In [11]:
execute_code_postgres("""GRANT SELECT
ON  empregado
TO  aluno01""")

GRANT
>>>>>>>    Processing performed correctly     <<<<<<
Database connection closed.


# Acessando a tabela empregado

#### Uma vez que as permissões foram atribuídas podemos acessar a tabela conforme observadom

In [12]:
execute_code_aluno01("""select * from empregado""")

SELECT 8
>>>>>>>    Processing performed correctly     <<<<<<
Database connection closed.


# DROP ROLE role_name;
Possibilita apagar um grupo ou usuário de acesso do sistema.

In [13]:
execute_code_postgres(""" drop role aluno01
""")

role "aluno01" cannot be dropped because some objects depend on it
DETAIL:  privileges for table empregado

>>>>>>>   Processing ERROR   <<<<<<
Database connection closed.


> Observe que neste caso precisamos antes remover as permissoes definidas para o usuario aluno01, pois este usuário tem permissões relacionadas a tabela empregado.

# Removendo permissões do usuário aluno01 relativas a tabela empregado.

In [14]:
execute_code_postgres(""" 
revoke select 
on empregado 
from aluno01
""")

REVOKE
>>>>>>>    Processing performed correctly     <<<<<<
Database connection closed.


# DROP ROLE role_name;
Possibilita apagar um grupo ou usuário de acesso do sistema.

In [15]:
execute_code_postgres(""" drop role aluno01
""")

DROP ROLE
>>>>>>>    Processing performed correctly     <<<<<<
Database connection closed.


### Pronto, agora o usuário/role aluno01 foi excluído do sgbd.